<a href="https://colab.research.google.com/github/ElizavetaNosova/CompLing_homeworks/blob/master/Topicmodeling_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install pandas
!pip install nltk
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.3MB/s 
     |████████████████████████████████| 256kB 38.5MB/s 
  Created wheel for DAWG: filename=DAWG-0.7.8-cp36-cp36m-linux_x86_64.whl size=771493 sha256=0ad8b821482700ae370c9fa9469f715775a7957ec7b39e66848628e310a935d6
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 39.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=a9c697fe3c03370cf836c26327d7d2a170e7640fb1758e03ffea8d66f7e9fefa
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=b223e7455751226f1c1432dbef75f504d8f1217e5f523a16fdf913bb1e097936
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'бол

При нормализации сразу не будем включать слова из списка стоп-слов и другие слова служебных частей речи, местоимения (видимо, местоимения-существительные, так как pymorphy только их выделяет отдельно) и числительные.
Полагаю, ято в связи с этим список стопслов можно изначально урезать: скопировать из nltk и оставить только другие части речи, раз вышеупомянутые всё равно будут проверяться отдельно.
Впрочем, непонятно, зачем я это делаю, если эти слова всё равно вряд ли станут "ключевыми" для тем

In [0]:
stops = ['как','так', 'было', 'еще', 'нет', 'теперь', 'когда', 'вдруг', 'уже', 'быть', 'был', 'нибудь', 'опять', 'там', 'потом', 'себя', 'ничего', 'может',  'тут', 'где', 'есть', 'надо', 'чем', 'была', 'сам', 'раз', 'тоже', 'себе', 'под', 'будет', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве',  'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']

def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, n):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(n) if morph.parse(word)[0].tag.POS not in ['NUMR', 'NPRO', 'PREP', 'CONJ', 'PRCL', 'INTJ']}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [8]:
from google.colab import files

uploaded = files.upload()

Saving habr_texts.txt to habr_texts.txt


In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

In [22]:
print(len(texts))

4121


Подберём параметры, при которых когерентность будет наибольшей. Для начала поберём параметры нграм и словаря при каких-нибудь параметрах модели. Будем надеяться, что при других параметрах модели они тоже хорошие

In [26]:
ngram_threshold_values = [0.2, 0.4, 0.6, 0.8]
no_above_values = [0, 0.05, 0.1, 0.2, 0.25]
no_below_values = [10, 20, 30, 50]
best_res = 0
best_params = {}
for ngram_threshold in ngram_threshold_values:
  for no_above in no_above_values:
    for no_below in no_below_values:
      try:
        ph = gensim.models.Phrases(texts, scoring='npmi', threshold=ngram_threshold) # threshold можно подбирать
        p = gensim.models.phrases.Phraser(ph)
        ngrammed_texts = p[texts]
        dictinary = gensim.corpora.Dictionary(texts)
        dictinary.filter_extremes(no_above=no_above, no_below=no_below)
        dictinary.compactify()
        corpus = [dictinary.doc2bow(text) for text in texts]
        lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=0)
        coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                    texts=texts, 
                                                     dictionary=dictinary, coherence='c_v')
        if coherence_model_lda.get_coherence() > best_res:
          best_res = coherence_model_lda.get_coherence()
          best_params['ngram_threshold'] = ngram_threshold
          best_params['no_above'] = no_above
          best_params['no_below'] = no_below
      except Exception as e:
         print(e)
print(best_params)


cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)
cannot compute LDA over an empty collection (no terms)


Process ForkPoolWorker-56:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamulticore.py", line 333, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py", line 725, in do_estep
    gamma, sstats = self.inference(chunk, collect_sstats=True)
  File "/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py", line 679, in inference
    phinorm = np.dot(expElogthetad, expElogbetad) + eps
  File "<__array_function__ internals>", line 6, in dot
KeyboardInterrupt


KeyboardInterrupt: ignored

Если честно, все варианты я не перебрала, но через какое-то время (часа полтора) лучшим результатом было {'ngram_threshold': 0.8, 'no_above': 0.05, 'no_below': 10}, когерентность 0.3816700738434562.
Как минимум все предложенные характеристики нграм хотя бы по разу были опробованы, раз дело дошло до 0.8
 Промежуточные выводы: 
1. некоторые параметры отсекают весь корпус (отлавливалась соответствующая ошибка)
2. не надо перебирать 80 сочетаний характеристик, если я хочу, чтобы процесс закончился через адекватное время

In [28]:
print(best_params) 
print(best_res)

{'ngram_threshold': 0.8, 'no_above': 0.05, 'no_below': 10}
0.3816700738434562


С учётом второго вывода  подберём настройки модели. Очень хочется верить, что, если корпус будет оформлен один раз, процесс будет хоть немного быстрее.

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.8) 
p = gensim.models.phrases.Phraser(ph)
nrammed_texts = p[texts]
dictinary = gensim.corpora.Dictionary(texts)
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()
corpus = [dictinary.doc2bow(text) for text in texts]

In [0]:
num_topics = [50, 100, 200]
n_eval = [0,1]
best_res = 0
best_params = {}
best_topics = ''
for n_topics in num_topics:
  for n in n_eval:
      lda = gensim.models.LdaMulticore(corpus, n_topics, id2word=dictinary, eval_every=n)
      coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                texts=texts, 
                                                  dictionary=dictinary, coherence='c_v')
      if coherence_model_lda.get_coherence() > best_res:
        best_res = coherence_model_lda.get_coherence()
        best_params['topics'] = n_topics
        best_params['eval'] = n
        best_model = lda

In [37]:
best_model.print_topics()

[(34,
  '0.019*"инцидент" + 0.013*"фоновый" + 0.011*"client" + 0.010*"chrome" + 0.009*"вкладка" + 0.009*"учётный" + 0.008*"бюджет" + 0.007*"иб" + 0.007*"долг" + 0.006*"select"'),
 (80,
  '0.034*"twitter" + 0.019*"твит" + 0.018*"бот" + 0.012*"аккаунт" + 0.008*"выборка" + 0.008*"ботнет" + 0.008*"тестировщик" + 0.007*"общественный" + 0.006*"северный" + 0.006*"streaming"'),
 (38,
  '0.036*"боль" + 0.032*"китай" + 0.015*"правительство" + 0.015*"атрибут" + 0.015*"vpn" + 0.009*"сопротивление" + 0.008*"•" + 0.007*"lt;/summary&gt" + 0.007*"чиновник" + 0.007*"англ"'),
 (66,
  '0.014*"клетка" + 0.009*"атрибут" + 0.007*"математика" + 0.007*"женщина" + 0.006*"аудио" + 0.006*"образование" + 0.006*"ген" + 0.005*"хакер" + 0.005*"ит" + 0.005*"семья"'),
 (98,
  '0.026*"foreach" + 0.019*"итерация" + 0.018*"do" + 0.018*"apply" + 0.014*"вектор" + 0.013*"fun" + 0.009*"матрица" + 0.009*"repeat" + 0.008*"result" + 0.008*"margin"'),
 (61,
  '0.040*"домен" + 0.012*"переадресация" + 0.012*"миграция" + 0.008*"seo

Лучшие темы:
- ситуация в Китае (даже если убрать Китай, сочетание правительства и боли прекрасно)
(38,
  '0.036*"боль" + 0.032*"китай" + 0.015*"правительство" + 0.015*"атрибут" + 0.015*"vpn" + 0.009*"сопротивление" + 0.008*"•" + 0.007*"lt;/summary&gt" + 0.007*"чиновник" + 0.007*"англ"'),

- экраны и графика:
 '0.020*"яркость" + 0.014*"стикер" + 0.014*"репликация" + 0.007*"гаджет" + 0.006*"no" + 0.006*"more" + 0.005*"атрибут" + 0.005*"координата" + 0.005*"выделение" + 0.005*"цветок"'),

 - биоинформатика
 51,
  '0.014*"клетка" + 0.012*"временный" + 0.007*"ген" + 0.006*"php" + 0.005*"виртуализация" + 0.005*"broadcast" + 0.005*"прерывание" + 0.005*"малый" + 0.005*"образование" + 0.004*"параллельный"'),
 



В выдачу попадало много слов типа "user" и команд языков программирования. В принципе, это не помешало, но перед следующим этапом лучше эти слова исключить. Добавим в предобработку ограничение: будем учитывать только слова, написанные кириллицей.
Ещё попало имя "Андрей". В идеале стоило бы скопировать словарь имён и стереть оттуда имена известных чат-ботов и т.п., но будем надеяться, что корреляция имён с темами маловероятна.

In [0]:
def remove_english(text):
  new_text = re.sub('[a-zA-Z]+', '', text)
  return new_text

In [0]:
texts = open('habr_texts.txt').read().splitlines()
texts = [remove_english(remove_tags(text.lower())) for text in texts]
texts = opt_normalize(texts, 30000)

Добавим tfidf

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf = gensim.models.TfidfModel(id2word=dictionary)

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.8) 
p = gensim.models.phrases.Phraser(ph)
nrammed_texts = p[texts]
dictinary = gensim.corpora.Dictionary(texts)
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()
corpus = [dictinary.doc2bow(text) for text in texts]

In [26]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)

NameError: ignored

In [0]:
tfidf = gensim.models.TfidfModel(corpus)

In [0]:
num_topics = [50, 100, 200]
n_eval = [0,1]
best_res = 0
best_params = {}
best_topics = ''
for n_topics in num_topics:
  for n in n_eval:
      lda = gensim.models.LdaMulticore(tfidf[corpus], n_topics, id2word=dictinary, eval_every=n)
      coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                texts=texts, 
                                                  dictionary=dictinary, coherence='c_v')
      if coherence_model_lda.get_coherence() > best_res:
        best_res = coherence_model_lda.get_coherence()
        best_params['topics'] = n_topics
        best_params['eval'] = n
        best_model = lda

In [24]:
best_model.print_topics()

[(28,
  '0.697*"•" + 0.233*"±" + 0.019*"‘" + 0.012*"é" + 0.009*"́" + 0.006*"á" + 0.005*">" + 0.005*"`" + 0.003*"\u200a" + 0.002*"„"'),
 (20,
  '0.515*"‘" + 0.290*"`" + 0.007*"₽" + 0.007*"↑" + 0.007*"́" + 0.007*"ó" + 0.007*"±" + 0.007*"←" + 0.007*"é" + 0.007*"\u200b"'),
 (34,
  '0.780*"é" + 0.054*"€" + 0.044*"©" + 0.038*"№" + 0.024*"°" + 0.020*"▍" + 0.011*"\u200a" + 0.007*"£" + 0.005*"±" + 0.001*"·"'),
 (18,
  '0.268*"\xad" + 0.182*"²" + 0.159*"€" + 0.159*"`" + 0.064*"°" + 0.016*"‘" + 0.008*"№" + 0.006*"±" + 0.006*"́" + 0.006*"\u200b"'),
 (30,
  '0.374*"ó" + 0.234*">" + 0.143*"\u200b" + 0.109*"„" + 0.080*"№" + 0.032*"←" + 0.010*"^" + 0.006*"́" + 0.003*"`" + 0.000*"é"'),
 (19,
  '0.551*">" + 0.232*"`" + 0.103*"№" + 0.034*"^" + 0.019*"▍" + 0.019*"±" + 0.012*"←" + 0.007*"€" + 0.004*"£" + 0.004*"₽"'),
 (2,
  '0.536*"№" + 0.151*"±" + 0.119*"`" + 0.065*"°" + 0.005*"é" + 0.005*"↑" + 0.005*"́" + 0.005*"ó" + 0.005*"₽" + 0.005*"←"'),
 (33,
  '0.747*"é" + 0.046*"←" + 0.045*"‘" + 0.034*"`" + 0.028*

Модель решила ориентироваться на символы. Причём даже не на буквы, а просто на все подряд символы. Причём ей было очень грустно, так как некоторые символы в неё попали с весом 0
'0.950*"é" + 0.042*"•" + 0.001*"‘" + 0.001*"\u2003" + 0.000*"←" + 0.000*"↑" + 0.000*"́" + 0.000*"ó" + 0.000*"²" + 0.000*"±"'),
Мне эти тематические модели совершенно не нравятся

Воспользуемся NMF

In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
stexts = [' '.join(text) for text in texts]

In [0]:
import math
min_error = math.inf
for max_f in [500, 1000]:
  vectorizer = TfidfVectorizer(max_features=max_f)
  X = vectorizer.fit_transform(stexts)
  for a in [0, 1]:
    for n in [20, 40]:
      model = NMF(n_components=n, alpha=a)
      model.fit(X)
      error = model.reconstruction_err_
      if error < min_error:
        min_error = error
        best_model = model


In [18]:
top_words = best_model.components_.argsort()[:,:-5:-1]

feat_names = vectorizer.get_feature_names()

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 наш  литр  метр  далее
1 далее  литр  наш  плата
2 метр  далее  год  литр
3 плата  год  метр  наш
4 год  плата  литр  метр
5 плата  метр  наш  число
6 наш  число  плата  метр
7 наш  литр  год  число
8 далее  литр  наш  число
9 метр  далее  плата  число
10 год  число  литр  метр
11 плата  далее  год  метр
12 плата  наш  далее  метр
13 число  литр  метр  плата
14 год  число  плата  далее
15 число  метр  наш  плата
16 литр  метр  год  плата
17 метр  литр  плата  наш
18 год  наш  далее  метр
19 плата  метр  литр  число
20 литр  метр  наш  плата
21 наш  далее  метр  плата
22 год  плата  число  далее
23 число  литр  плата  наш
24 число  год  метр  плата
25 литр  плата  далее  наш
26 литр  год  число  плата
27 наш  литр  год  метр
28 плата  наш  число  метр
29 число  год  плата  далее
30 плата  литр  год  далее
31 далее  число  плата  метр
32 число  далее  наш  метр
33 метр  литр  плата  наш
34 литр  метр  наш  плата
35 плата  наш  метр  литр
36 наш  метр  год  далее
37 далее  наш  плата  ч

Получились ужасные темы, потому что tfidf большую часть слов решил не рассматривать. Когда я пыталась настраивать пороги (в том числе предлагала ему 0 и плюс бесконечность), он выражал недовольство, так как не мог найти ни одного текста ни с какими порогами. Ещё была выдача, состоящая исключительно из комбинаций слов "год" и "число". 